# <center>LogisticRegression</center>

<div class="alert alert-success">
    <b>Use this to install/upgrade your package</b>
    
#Update sklearn to prevent version mismatches<br>
!pip install sklearn --upgrade

#install joblib. This will be used to save your model.<br>
#Restart your kernel after installing<br>
!pip install joblib
</div>

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Read the CSV and Perform Basic Data Cleaning

In [3]:
df = pd.read_csv("../data/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [4]:
# Convert dtypes of int64 to float64
for column, content in df.items():
    if df[column].dtype == 'int64':
        df = df.astype({column: 'float64'})

In [5]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
#import joblib

# Assign data to X and y
X = df.drop("koi_disposition", axis=1)
y = df["koi_disposition"]

# Split data into training and testing groups
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [6]:
# Scale X values
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

#### Building and training the model

In [7]:
model_1 = LogisticRegression(solver='newton-cg', multi_class='auto')
model_1.fit(X_train_scaled, y_train)

model_1_training_score = round(model_1.score(X_train_scaled, y_train)*100,3)
base_accuracy = round(model_1.score(X_test_scaled, y_test)*100,3)

print(f"Training Data Score: {model_1_training_score} %")
print(f"Testing Data Score: {base_accuracy} %")

Training Data Score: 85.504 %
Testing Data Score: 86.213 %


#### Selecting the best 20 features

In [8]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [9]:
X_minmax = MinMaxScaler().fit(X_train)

X_train_minmax = X_minmax.transform(X_train)
X_test_minmax = X_minmax.transform(X_test)

In [10]:
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X_train_minmax,y_train)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
top_features = featureScores.nlargest(20,'Score')
print(top_features)  #print best features

                Specs        Score
1       koi_fpflag_ss  1148.814143
2       koi_fpflag_co  1039.969284
0       koi_fpflag_nt   741.439148
3       koi_fpflag_ec   658.724200
26      koi_model_snr    93.458602
16          koi_depth    73.766986
27   koi_tce_plnt_num    70.909311
11    koi_impact_err1    33.022438
4          koi_period    24.758707
5     koi_period_err1    19.320861
29     koi_steff_err1    19.018835
22            koi_teq    18.660398
14  koi_duration_err1    16.470717
37                 ra    14.497078
32     koi_slogg_err1    12.906484
8    koi_time0bk_err1    11.611191
13       koi_duration     8.355489
38                dec     7.480447
35      koi_srad_err1     7.173358
28          koi_steff     4.580405


In [11]:
feature_arrey = []
for i in top_features["Specs"]:
    feature_arrey.append(i)

In [12]:
feature_selected_df = df.loc[:, feature_arrey]
feature_selected_df.head()

,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_nt,koi_fpflag_ec,koi_model_snr,koi_depth,koi_tce_plnt_num,koi_impact_err1,koi_period,koi_period_err1,koi_steff_err1,koi_teq,koi_duration_err1,ra,koi_slogg_err1,koi_time0bk_err1,koi_duration,dec,koi_srad_err1,koi_steff
0,0.0,0.0,0.0,0.0,25.8,874.8,2.0,0.059,54.418383,2.479000e-04,81.0,443.0,0.11600,291.93423,0.064,0.003520,4.50700,48.141651,0.105,5455.0
1,1.0,0.0,0.0,0.0,76.3,10829.0,1.0,5.126,19.899140,1.490000e-05,158.0,638.0,0.03410,297.00482,0.044,0.000581,1.78220,48.134129,0.233,5853.0
2,1.0,0.0,0.0,0.0,505.6,8079.2,1.0,0.115,1.736952,2.630000e-07,157.0,1395.0,0.00537,285.53461,0.053,0.000115,2.40641,48.285210,0.201,5805.0
3,0.0,0.0,0.0,0.0,40.9,603.3,1.0,0.235,2.525592,3.760000e-06,169.0,1406.0,0.04200,288.75488,0.070,0.001130,1.65450,48.226200,0.334,6031.0
4,0.0,0.0,0.0,0.0,40.2,686.0,2.0,0.139,4.134435,1.050000e-05,189.0,1160.0,0.06730,296.28613,0.054,0.001900,3.14020,48.224670,0.315,6046.0


In [13]:
X_feature = feature_selected_df.copy()
y_feature = df["koi_disposition"]

In [14]:
X_train_feature, X_test_feature, y_train_feature, y_test_feature = train_test_split(X_feature, y_feature, random_state=42)
X_train_feature.head()

,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_nt,koi_fpflag_ec,koi_model_snr,koi_depth,koi_tce_plnt_num,koi_impact_err1,koi_period,koi_period_err1,koi_steff_err1,koi_teq,koi_duration_err1,ra,koi_slogg_err1,koi_time0bk_err1,koi_duration,dec,koi_srad_err1,koi_steff
6122,0.0,0.0,0.0,0.0,10.8,123.1,1.0,0.305,6.768901,7.380000e-05,154.0,1017.0,0.3060,294.40472,0.153,0.008440,3.61600,39.351681,0.310,5737.0
6370,1.0,0.0,0.0,1.0,13.8,114.6,1.0,0.193,0.733726,6.060000e-06,158.0,1867.0,0.2820,284.50391,0.033,0.007950,2.30900,42.463860,0.211,5855.0
2879,0.0,0.0,1.0,0.0,254.3,641.1,1.0,0.879,7.652707,6.540000e-05,151.0,989.0,0.0000,295.50211,0.050,0.006190,79.89690,38.983540,0.290,6328.0
107,0.0,0.0,0.0,0.0,38.4,875.4,1.0,0.145,7.953547,1.910000e-05,76.0,696.0,0.0595,291.15878,0.056,0.001820,2.63120,40.750271,0.023,4768.0
29,0.0,0.0,0.0,0.0,696.5,9802.0,1.0,0.016,4.959319,5.150000e-07,77.0,1103.0,0.0075,292.16705,0.110,0.000083,2.22739,48.727589,0.173,5712.0


In [15]:
X_minmax_feature = MinMaxScaler().fit(X_train_feature)

X_train_minmax_feature = X_minmax_feature.transform(X_train_feature)
X_test_minmax_feature = X_minmax_feature.transform(X_test_feature)

In [16]:
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
                   solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [18]:
classifier.fit(X_train_minmax_feature, y_train_feature)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
classifier_score_feature = round(classifier.score(X_train_minmax_feature, y_train_feature)*100, 2)
base_model_score_feature = round(classifier.score(X_test_minmax_feature, y_test_feature)*100, 2)

print(f'Train Data Score: {classifier_score_feature}%')
print(f'Test Data Score: {base_model_score_feature}%')

Train Data Score: 84.59%
Test Data Score: 84.38%


#### Tuning the model

In [20]:
# Create the GridSearchCV model
model_3 = LogisticRegression(solver='newton-cg', multi_class='auto')

param_grid = {
    'C': np.logspace(0, 4, 10),
    'penalty': ['l2']
}
grid = GridSearchCV(model_3, param_grid, cv=5, verbose=0)

# Train the model with GridSearch
_ = grid.fit(X_train_feature, y_train_feature)

#### Train tuned model

In [21]:
# Tuned parameters
C = grid.best_params_['C']
penalty = grid.best_params_['penalty']

# Tuned model
tuned_model = LogisticRegression(solver='newton-cg', multi_class='auto',
                                 C=C, penalty=penalty)
tuned_model.fit(X_train_feature, y_train_feature)

model_3_training_score = round(tuned_model.score(X_train_feature, y_train_feature)*100,3)
tuned_accuracy = round(tuned_model.score(X_test_feature, y_test_feature)*100,3)

print(f"Training Data Score: {model_3_training_score} %")
print(f"Testing Data Score: {tuned_accuracy} %")

Training Data Score: 88.175 %
Testing Data Score: 87.128 %


#### Evaluation

In [22]:
evaluations = {'': ['Base Model', 'Select Features Model', 'Tuned Model'],
               'Accuracy': [f"{base_accuracy}%", f"{base_model_score_feature}%", f"{tuned_accuracy}%"]}

evaluations_df = pd.DataFrame(evaluations)
evaluations_df = evaluations_df.set_index('')

#evaluations_df.to_csv('../Resources/LogisticRegression_eval.csv')
evaluations_df

,Accuracy
,
Base Model,86.213%
Select Features Model,84.38%
Tuned Model,87.128%


#### Save the model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = '.sav'
joblib.dump(your_model, filename)

# Select your features (columns)

In [ ]:
# Set features. This will also be used as your x values.
selected_features = df[['names', 'of', 'selected', 'features', 'here']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [ ]:
X_train.head()

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [ ]:
# Scale your data

# Train the Model



In [ ]:
print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model

In [ ]:
# Train the model with GridSearch

In [ ]:
print(grid2.best_params_)
print(grid2.best_score_)

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)